# Linear regression: closed form vs. optimization (GD/SGD)

This notebook walks through:

1. Load a scikit-learn dataset with continuous features  
2. Fit a **1D** linear regression line using an **exact (closed-form) solver**  
3. Compute **L1** and **L2** losses  
4. Plot **loss vs. parameter(s)** to see **V-shape (L1)** and **parabola (L2)** behavior  
5. Extend to **multiple features** (design matrix)  
6. Solve with **OLS closed form** and **time** it (why it becomes challenging at scale)  
7. Solve with **Gradient Descent (GD)** and **Stochastic GD (SGD)** for both 1D and multi-D  
8. Plot **loss vs. parameters** (and loss vs. iterations)  
9. Visualize the multi-D surface for **two features** (3D + contour), and optionally **animate** GD/SGD paths


In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler

import time
from dataclasses import dataclass

np.random.seed(42)
plt.rcParams["figure.figsize"] = (8, 5)


## Notation (linear algebra)

For a dataset with **n** examples and **p** features:

- Design matrix:  
  $X \in \mathbb{R}^{n \times p}$
- Targets:  
  $y \in \mathbb{R}^{n}$
- Linear model (with intercept):
  $\hat y = b\mathbf{1} + Xw$
  where $w \in \mathbb{R}^{p}$ and $b \in \mathbb{R}$.

Common losses:

- **L2 / MSE (sum of squares)**:
  $\mathcal{L}_2(w,b) = \sum_{i=1}^{n} (y_i - (b + x_i^\top w))^2$
- **L1 / MAE (sum of absolute errors)**:
  $\mathcal{L}_1(w,b) = \sum_{i=1}^{n} \left|y_i - (b + x_i^\top w)\right|$

Closed-form **OLS** solution (when \(X^\top X\) is invertible):

$w^* = (X^\top X)^{-1}X^\top y$

A numerically safer equivalent is solving the linear system:
$(X^\top X)w = X^\top y$
without explicitly forming $(X^\top X)^{-1}$.


In [ ]:
# 1) Load a dataset with continuous, standardized features
diabetes = load_diabetes()
X = diabetes.data          # already standardized in sklearn diabetes
y = diabetes.target.astype(float)

feature_names = diabetes.feature_names
n, p = X.shape
n, p, feature_names[:5], y[:3]


## 2) One feature → one target: fit a line with an exact solver

Pick a single feature \(x \in \mathbb{R}^n\) and fit:

\[
\hat y = b + wx
\]

We can solve for \(w,b\) by the normal equations in 1D. A convenient way:
- augment the design matrix with a column of ones,
- solve the 2-parameter OLS problem.


In [ ]:
# Choose one feature (e.g., BMI) and create a 1D design matrix with intercept
j = feature_names.index("bmi") if "bmi" in feature_names else 2
x = X[:, j]  # (n,)

X1 = np.column_stack([np.ones(n), x])  # (n,2): [1, x]
# Closed-form OLS for (b, w): theta = (X^T X)^{-1} X^T y
theta_1d = np.linalg.solve(X1.T @ X1, X1.T @ y)  # safer than explicit inverse
b_hat_1d, w_hat_1d = theta_1d

b_hat_1d, w_hat_1d


In [ ]:
# Plot the data and fitted line
y_hat_1d = X1 @ theta_1d

plt.figure()
plt.scatter(x, y, alpha=0.6, label="data")
xs = np.linspace(x.min(), x.max(), 200)
plt.plot(xs, b_hat_1d + w_hat_1d * xs, linewidth=2, label="OLS fit")
plt.xlabel(f"x = {feature_names[j]}")
plt.ylabel("y (target)")
plt.title("1D linear regression with closed-form OLS")
plt.legend()
plt.grid(True)
plt.show()


## 3) Compute L1 and L2 loss (for this fitted line)

We'll compute:

\[
\mathcal{L}_1 = \sum_i |y_i - \hat y_i|,\qquad
\mathcal{L}_2 = \sum_i (y_i - \hat y_i)^2
\]


In [ ]:
resid_1d = y - y_hat_1d

L1_1d = np.sum(np.abs(resid_1d))
L2_1d = np.sum(resid_1d**2)

L1_1d, L2_1d


## 4) Plot L1 and L2 vs parameters (show V-shape vs parabola)

To make the shapes clear, we will plot the loss as a function of **a single parameter** $w$.
To keep it 1D, we temporarily fix the intercept $b$ to the OLS value $b=\hat b$, and vary $w$.

$\mathcal{L}_1(w) = \sum_i |y_i - (\hat b + w x_i)|,\qquad
\mathcal{L}_2(w) = \sum_i (y_i - (\hat b + w x_i))^2$

- $\mathcal{L}_1(w)$ is **piecewise linear** → "V-ish" around its minimum  
- $\mathcal{L}_2(w)$ is **quadratic** → parabola around its minimum


In [ ]:
# Sweep w around the OLS solution and compute L1/L2
w_grid = np.linspace(w_hat_1d - 80, w_hat_1d + 80, 600)

L1_grid = []
L2_grid = []
for w in w_grid:
    y_hat = b_hat_1d + w * x
    r = y - y_hat
    L1_grid.append(np.sum(np.abs(r)))
    L2_grid.append(np.sum(r**2))

L1_grid = np.array(L1_grid)
L2_grid = np.array(L2_grid)

fig, ax = plt.subplots()
ax.plot(w_grid, L1_grid, label="L1 loss (sum abs)")
ax.axvline(w_hat_1d, linestyle="--", label="ŵ (OLS)")
ax.set_xlabel("w")
ax.set_ylabel("Loss")
ax.set_title("L1 loss vs parameter w (V-shaped / piecewise linear)")
ax.grid(True)
ax.legend()
plt.show()

fig, ax = plt.subplots()
ax.plot(w_grid, L2_grid, label="L2 loss (sum squares)")
ax.axvline(w_hat_1d, linestyle="--", label="ŵ (OLS)")
ax.set_xlabel("w")
ax.set_ylabel("Loss")
ax.set_title("L2 loss vs parameter w (parabola / quadratic)")
ax.grid(True)
ax.legend()
plt.show()


## 5) Multiple features → design matrix

Now we use multiple features and solve:

$\hat y = b + Xw$

We'll use **all features** from the diabetes dataset first, then later pick only 2 features for a 3D visualization.


In [ ]:
# Add intercept column for multi-feature OLS
X_full = np.column_stack([np.ones(n), X])  # (n, p+1); parameters theta = [b, w1, ..., wp]

# Closed-form OLS solve: (X^T X) theta = X^T y
theta_full = np.linalg.solve(X_full.T @ X_full, X_full.T @ y)
b_hat_full, w_hat_full = theta_full[0], theta_full[1:]

b_hat_full, w_hat_full[:5], w_hat_full.shape


## 6) Timing OLS closed-form (why it can be challenging)

The bottleneck in closed-form OLS is typically forming and solving the **normal equations**:

- Compute $X^\top X:O(np^2)$
- Solve a $p \times p$ system: roughly $O(p^3)$

For small \(p\) it's fast, but as \(p\) grows, \(p^3\) gets expensive.

Below we create **synthetic high-dimensional** feature matrices (still derived from the dataset) to show the cost scaling.
We also add a small ridge term $\lambda I$ to make the system well-conditioned for inversion/solve:

$(X^\top X + \lambda I)\theta = X^\top y$


In [ ]:
def time_closed_form(n=442, p=50, ridge=1e-6, repeats=3):
    # synthetic X: random projection of original X to p dims
    # (so we're not just padding zeros)
    X0 = X  # (n,10)
    Wproj = np.random.randn(X0.shape[1], p)
    Xp = X0 @ Wproj  # (n,p)

    X_aug = np.column_stack([np.ones(n), Xp])  # (n, p+1)
    A = X_aug.T @ X_aug
    A = A + ridge * np.eye(A.shape[0])
    b = X_aug.T @ y

    # time solve
    times = []
    for _ in range(repeats):
        t0 = time.perf_counter()
        _ = np.linalg.solve(A, b)
        t1 = time.perf_counter()
        times.append(t1 - t0)
    return float(np.mean(times)), float(np.std(times))

dims = [10, 50, 100, 200, 400, 800]
timings = []
for p_dim in dims:
    mu, sd = time_closed_form(n=n, p=p_dim, ridge=1e-6, repeats=5)
    timings.append((p_dim, mu, sd))

timings


In [ ]:
# Plot timing vs dimension
dims_arr = np.array([t[0] for t in timings])
mu_arr = np.array([t[1] for t in timings])
sd_arr = np.array([t[2] for t in timings])

plt.figure()
plt.plot(dims_arr, mu_arr, marker="o")
plt.fill_between(dims_arr, mu_arr - sd_arr, mu_arr + sd_arr, alpha=0.2)
plt.xlabel("Number of synthetic features p")
plt.ylabel("Solve time (seconds)")
plt.title("Closed-form solve time grows quickly with p (normal equations)")
plt.grid(True)
plt.show()


## 7) Gradient Descent (GD) and Stochastic GD (SGD)

We'll optimize the **MSE objective** (mean squared error), which is differentiable:

\[
J(\theta) = \frac{1}{n}\|y - X\theta\|_2^2 = \frac{1}{n}(y - X\theta)^\top (y - X\theta)
\]

Gradient:
\[
\nabla_\theta J(\theta) = -\frac{2}{n}X^\top (y - X\theta)
\]

Update rule (GD):
\[
\theta^{(t+1)} = \theta^{(t)} - \eta \nabla_\theta J(\theta^{(t)})
\]

For SGD, we use a (mini)batch of size \(m\) (often 1):
\[
\theta^{(t+1)} = \theta^{(t)} - \eta \left(-\frac{2}{m}X_B^\top (y_B - X_B\theta^{(t)})\right)
\]


In [ ]:
@dataclass
class GDHistory:
    thetas: list
    losses: list

def mse_loss(Xa, y, theta):
    r = y - Xa @ theta
    return float(np.mean(r**2))

def gd_mse(Xa, y, lr=0.05, steps=200, theta0=None):
    n = Xa.shape[0]
    theta = np.zeros(Xa.shape[1]) if theta0 is None else theta0.astype(float).copy()
    hist = GDHistory(thetas=[theta.copy()], losses=[mse_loss(Xa, y, theta)])
    for _ in range(steps):
        r = y - Xa @ theta
        grad = -(2/n) * (Xa.T @ r)
        theta = theta - lr * grad
        hist.thetas.append(theta.copy())
        hist.losses.append(mse_loss(Xa, y, theta))
    return hist

def sgd_mse(Xa, y, lr=0.05, steps=500, batch_size=1, theta0=None, shuffle=True):
    n = Xa.shape[0]
    theta = np.zeros(Xa.shape[1]) if theta0 is None else theta0.astype(float).copy()
    hist = GDHistory(thetas=[theta.copy()], losses=[mse_loss(Xa, y, theta)])
    idx = np.arange(n)
    for t in range(steps):
        if shuffle and (t % n == 0):
            np.random.shuffle(idx)
        # mini-batch
        start = (t * batch_size) % n
        batch_idx = idx[start:start+batch_size]
        Xb = Xa[batch_idx]
        yb = y[batch_idx]
        rb = yb - Xb @ theta
        grad = -(2/len(batch_idx)) * (Xb.T @ rb)
        theta = theta - lr * grad
        hist.thetas.append(theta.copy())
        hist.losses.append(mse_loss(Xa, y, theta))  # track full-dataset loss for comparability
    return hist


### 7A) Solve the 1D problem with GD and SGD

We'll reuse the 1D augmented matrix \(X_1 = [\mathbf{1}, x]\) and compare to the OLS solution.


In [ ]:
# 1D optimization
hist_gd_1d = gd_mse(X1, y, lr=0.2, steps=80)
hist_sgd_1d = sgd_mse(X1, y, lr=0.05, steps=1200, batch_size=1)

plt.figure()
plt.plot(hist_gd_1d.losses, label="GD (full batch)")
plt.plot(hist_sgd_1d.losses, label="SGD (batch=1)", alpha=0.8)
plt.yscale("log")
plt.xlabel("Iteration")
plt.ylabel("MSE (log scale)")
plt.title("1D: GD vs SGD convergence (MSE)")
plt.grid(True)
plt.legend()
plt.show()

# Compare final parameters
theta_gd_1d = hist_gd_1d.thetas[-1]
theta_sgd_1d = hist_sgd_1d.thetas[-1]
theta_ols_1d = theta_1d

theta_ols_1d, theta_gd_1d, theta_sgd_1d


## 8–9) Graph the loss vs parameters (1D is easy)

For the 1D model (with intercept), parameters are \((b,w)\).  
We can visualize the loss surface by making a grid over \(b\) and \(w\) and plotting contours.

This also lets us overlay the **optimization trajectories** for GD and SGD.


In [ ]:
# Build a grid over (b, w) and compute MSE
b_grid = np.linspace(b_hat_1d - 80, b_hat_1d + 80, 200)
w_grid2 = np.linspace(w_hat_1d - 80, w_hat_1d + 80, 200)

BB, WW = np.meshgrid(b_grid, w_grid2)
Loss = np.zeros_like(BB)

for i in range(BB.shape[0]):
    b0 = BB[i]
    w0 = WW[i]
    y_hat = b0 + w0 * x
    Loss[i] = np.mean((y - y_hat)**2)

plt.figure(figsize=(7,6))
cs = plt.contour(BB, WW, Loss, levels=30)
plt.clabel(cs, inline=True, fontsize=8)
plt.xlabel("b (intercept)")
plt.ylabel("w (slope)")
plt.title("1D MSE loss contours over (b, w)")
plt.grid(True)

# overlay paths
gd_path = np.array(hist_gd_1d.thetas)  # (T,2) with [b,w]
sgd_path = np.array(hist_sgd_1d.thetas)

plt.plot(gd_path[:,0], gd_path[:,1], marker="o", markersize=2, linewidth=1.5, label="GD path")
plt.plot(sgd_path[::20,0], sgd_path[::20,1], marker="x", markersize=3, linewidth=1.0, label="SGD path (every 20 steps)")

plt.scatter([theta_ols_1d[0]], [theta_ols_1d[1]], s=80, label="OLS optimum")
plt.legend()
plt.show()


## 10) Multi-dimensional: visualize only 2 features

With \(p>2\), the loss surface lives in high dimensions and can't be fully graphed.

A common trick is to:
- pick 2 features (\(w_1,w_2\)),
- optionally fix the intercept and other weights,
- then visualize the surface over \((w_1,w_2)\).

We'll pick two features and fit a model with intercept + these two weights:
\[
\hat y = b + w_1 x_{(1)} + w_2 x_{(2)}
\]


In [ ]:
# Pick two features for visualization
feat_idx = [feature_names.index("bmi") if "bmi" in feature_names else 2,
            feature_names.index("bp")  if "bp"  in feature_names else 3]
f1, f2 = feat_idx
X2 = X[:, [f1, f2]]
X2_aug = np.column_stack([np.ones(n), X2])  # (n,3): [1, x1, x2]

theta_ols_2 = np.linalg.solve(X2_aug.T @ X2_aug, X2_aug.T @ y)
theta_ols_2


In [ ]:
# Run GD/SGD on this 2-feature problem
hist_gd_2 = gd_mse(X2_aug, y, lr=0.2, steps=120)
hist_sgd_2 = sgd_mse(X2_aug, y, lr=0.03, steps=2500, batch_size=1)

plt.figure()
plt.plot(hist_gd_2.losses, label="GD")
plt.plot(hist_sgd_2.losses, label="SGD", alpha=0.8)
plt.yscale("log")
plt.xlabel("Iteration")
plt.ylabel("MSE (log scale)")
plt.title("2-feature model: GD vs SGD convergence")
plt.grid(True)
plt.legend()
plt.show()


### 3D surface (optional) + contour with optimization paths

We'll visualize the MSE as a function of \((w_1,w_2)\) while fixing \(b\) to its OLS value \(\hat b\).


In [ ]:
b_fix = float(theta_ols_2[0])

w1_grid = np.linspace(theta_ols_2[1]-120, theta_ols_2[1]+120, 120)
w2_grid = np.linspace(theta_ols_2[2]-120, theta_ols_2[2]+120, 120)
W1, W2 = np.meshgrid(w1_grid, w2_grid)

Z = np.zeros_like(W1)
x1 = X2[:, 0]
x2 = X2[:, 1]
for i in range(W1.shape[0]):
    y_hat = b_fix + W1[i]*x1 + W2[i]*x2
    Z[i] = np.mean((y - y_hat)**2)

# Contour plot + paths projected to (w1,w2)
gd_path2 = np.array(hist_gd_2.thetas)[:, 1:3]
sgd_path2 = np.array(hist_sgd_2.thetas)[:, 1:3]

plt.figure(figsize=(7,6))
cs = plt.contour(W1, W2, Z, levels=30)
plt.clabel(cs, inline=True, fontsize=8)
plt.xlabel(f"w1 for {feature_names[f1]}")
plt.ylabel(f"w2 for {feature_names[f2]}")
plt.title("2-feature MSE contours (b fixed) with GD/SGD paths")
plt.grid(True)

plt.plot(gd_path2[:,0], gd_path2[:,1], marker="o", markersize=2, linewidth=1.5, label="GD path")
plt.plot(sgd_path2[::30,0], sgd_path2[::30,1], marker="x", markersize=3, linewidth=1.0, label="SGD path (every 30 steps)")
plt.scatter([theta_ols_2[1]], [theta_ols_2[2]], s=80, label="OLS optimum (w1,w2)")

plt.legend()
plt.show()


In [ ]:
# 3D surface plot (can be heavy; keep grid moderate)
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

fig = plt.figure(figsize=(9,6))
ax = fig.add_subplot(111, projection="3d")
ax.plot_surface(W1, W2, Z, rstride=2, cstride=2, linewidth=0, antialiased=True, alpha=0.8)
ax.set_xlabel(f"w1 ({feature_names[f1]})")
ax.set_ylabel(f"w2 ({feature_names[f2]})")
ax.set_zlabel("MSE (b fixed)")
ax.set_title("2-feature MSE surface (3D)")
plt.show()


## 11) Optional: animate GD vs SGD path on the contour (GIF)

If `Pillow` is available, we can save an animated GIF that shows the optimization path.

If this fails in your environment, you can skip this cell and rely on the static contour plots above.


In [ ]:
# Try to create a GIF animation of the paths (contour + moving points)

import matplotlib.animation as animation

gif_path = "gd_vs_sgd_paths.gif"

# Reduce the number of frames for a lighter GIF
gd_frames = gd_path2
sgd_frames = sgd_path2[::10]  # downsample SGD

T = max(len(gd_frames), len(sgd_frames))
# pad shorter path by repeating last point
if len(gd_frames) < T:
    gd_frames = np.vstack([gd_frames, np.repeat(gd_frames[-1][None,:], T-len(gd_frames), axis=0)])
if len(sgd_frames) < T:
    sgd_frames = np.vstack([sgd_frames, np.repeat(sgd_frames[-1][None,:], T-len(sgd_frames), axis=0)])

fig, ax = plt.subplots(figsize=(7,6))
cs = ax.contour(W1, W2, Z, levels=30)
ax.clabel(cs, inline=True, fontsize=8)
ax.set_xlabel(f"w1 for {feature_names[f1]}")
ax.set_ylabel(f"w2 for {feature_names[f2]}")
ax.set_title("GD vs SGD path (b fixed)")

# Static optimum marker
ax.scatter([theta_ols_2[1]], [theta_ols_2[2]], s=80, label="OLS optimum")

# Lines + moving points
(line_gd,) = ax.plot([], [], linewidth=1.5, label="GD")
(line_sgd,) = ax.plot([], [], linewidth=1.0, label="SGD")
(pt_gd,) = ax.plot([], [], marker="o", markersize=5, linestyle="None")
(pt_sgd,) = ax.plot([], [], marker="x", markersize=6, linestyle="None")

ax.legend()
ax.grid(True)

def init():
    line_gd.set_data([], [])
    line_sgd.set_data([], [])
    pt_gd.set_data([], [])
    pt_sgd.set_data([], [])
    return line_gd, line_sgd, pt_gd, pt_sgd

def update(frame):
    line_gd.set_data(gd_frames[:frame+1,0], gd_frames[:frame+1,1])
    line_sgd.set_data(sgd_frames[:frame+1,0], sgd_frames[:frame+1,1])
    pt_gd.set_data([gd_frames[frame,0]], [gd_frames[frame,1]])
    pt_sgd.set_data([sgd_frames[frame,0]], [sgd_frames[frame,1]])
    return line_gd, line_sgd, pt_gd, pt_sgd

anim = animation.FuncAnimation(fig, update, init_func=init, frames=T, interval=30, blit=True)

# Save GIF
try:
    writer = animation.PillowWriter(fps=25)
    anim.save(gif_path, writer=writer)
    print(f"Saved GIF to: {gif_path}")
except Exception as e:
    print("GIF save failed:", repr(e))
    print("You can still use the static plots above.")
plt.close(fig)


## Summary

- Closed-form OLS gives an exact solution (when feasible) but can become expensive as feature dimension \(p\) grows.
- GD/SGD avoid matrix inversion; they can scale better to large \(p\) and large \(n\), but require:
  - learning rate tuning,
  - iteration budgeting,
  - and careful monitoring of convergence.

You now have:
- L1 vs L2 loss curves (V-shape vs parabola)
- Closed-form OLS solutions (1D and multi-D)
- GD/SGD implementations from scratch
- Loss vs iterations plots
- Loss vs parameter(s) plots (contours / surfaces)
- Optional GIF animation of optimization paths
